# Assignment 2: algorithmic affordances
## Nadine Kanbier 4283724

In [1]:
# load the packages
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
movies = pd.read_csv('/Users/nadinekanbier/Desktop/Applied Data Science/Periode 3/Personalization/Assignment2/movies_metadata.csv')

In [4]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# 1. Simple recommender (best rated movies)

In [5]:
# Use weighted rating (explain in technical report)
C = movies['vote_average'].mean()
m = movies['vote_count'].quantile(0.90)

In [6]:
# new dataframe for qualified movies
q_movies = movies.copy().loc[movies['vote_count'] >= m]

In [7]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [9]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(8)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404


# 2. Content-based recommender (because you watched...)

In [10]:
movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [11]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
movies['overview'] = movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [12]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

['avails',
 'avaks',
 'avalanche',
 'avalanches',
 'avallone',
 'avalon',
 'avant',
 'avanthika',
 'avanti',
 'avaracious']

In [13]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [15]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [16]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies['title'].iloc[movie_indices]

In [17]:
get_recommendations ('Jumanji')

21633         Table No. 21
45253                 Quiz
41573         Snowed Under
35509             The Mend
44376    Liar Game: Reborn
17223       The Dark Angel
8801               Quintet
6166             Brainscan
30981         Turkey Shoot
9503             Word Wars
Name: title, dtype: object

# 3. Collaborative filtering (people like you liked...)

In [18]:
# First, I needed to link the movie dataset and ratings to eachother through the links file.

In [19]:
ratings = pd.read_csv('/Users/nadinekanbier/Desktop/Applied Data Science/Periode 3/Personalization/Assignment2/ratings.csv')
links = pd.read_csv('/Users/nadinekanbier/Downloads/archive (2)/links.csv')

In [20]:
links.rename(columns = {'imdbId':'imdb_id'}, inplace = True)

In [21]:
links['imdb_id'] = links['imdb_id'].astype(str)
movies['imdb_id'] = movies['imdb_id'].astype(str)

In [22]:
movies['imdb_id'] = movies['imdb_id'].str[2:]

In [23]:
linksmovies = pd.merge(movies, links, on="imdb_id")
df = pd.merge(linksmovies, ratings, on="movieId")

In [24]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,movieId,tmdbId,userId,rating,timestamp
0,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,103208,84198.0,24025,3.0,1459091571
1,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,103208,84198.0,44875,4.0,1496778600
2,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,103208,84198.0,45811,3.0,1499338522
3,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,103208,84198.0,51159,0.5,1486722233
4,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,103208,84198.0,53562,3.0,1401138155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012904,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,404604,5690142,hi,Maa,The bliss of a biology teacher’s family life i...,...,NaN,Mom,False,6.6,14.0,176249,404604.0,24523,3.5,1501626732
2012905,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,404604,5690142,hi,Maa,The bliss of a biology teacher’s family life i...,...,NaN,Mom,False,6.6,14.0,176249,404604.0,189323,3.5,1501737788
2012906,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,420346,4130180,en,The Morning After,The Morning After is a feature film that consi...,...,What happened last night?,The Morning After,False,4.0,2.0,176251,420346.0,235828,2.0,1501631121
2012907,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,Rising and falling between a man and woman,Subdue,False,4.0,1.0,176269,439050.0,97528,3.5,1501764806


In [25]:
# create a smaller selection since calculating is costly (and takes forever)
user_ratings = df[['userId', 'rating', 'title']].drop_duplicates(['userId', 'title'])

In [26]:
# one way to create smaller selection is by looking at average voting by a user

# count total reviews
avg_user = user_ratings.groupby(['userId']).size().reset_index(name='counts')

# set index for later
avg_user = avg_user.set_index('userId')

# select x-number of reviews
avg_user = avg_user[avg_user['counts'] > 50]

# user the index to create an list of ids that have more than x-number 
user_index = avg_user.index

# iloc (so look for the user id which is int) and make a smaller selection now including the review
#smaller_selection = user_ratings[user_ratings['user_id'] == users_index]

smaller_selection = user_ratings.loc[user_ratings['userId'].isin(user_index)]

In [27]:
#user_item matrix
user_ratings_pivot = smaller_selection.pivot(index='userId', columns='title', values='rating')
user_ratings_pivot.loc[43463].sort_values(ascending=False).head(20)

title
Cosmopolis                                   4.5
The Strange Color of Your Body's Tears       4.0
The Tale of the Princess Kaguya              4.0
It Follows                                   4.0
The Turin Horse                              4.0
Milk                                         4.0
Once Upon a Time in Anatolia                 4.0
Maleficent                                   4.0
A Girl Walks Home Alone at Night             4.0
Blancanieves                                 4.0
A Prophet                                    4.0
Oz: The Great and Powerful                   3.5
Behind the Candelabra                        3.5
Frances Ha                                   3.5
Incendies                                    3.5
The Ghost Writer                             3.5
Night Will Fall                              3.5
Cinema's Exiles: From Hitler to Hollywood    3.5
The Extraordinary Voyage                     3.5
Pontypool                                    3.5
Name: 43463, d

In [28]:
avg_ratings = user_ratings_pivot.mean(axis=1)
user_ratings_table = user_ratings_pivot
user_ratings_pivot = user_ratings_pivot.sub(avg_ratings, axis=0)

In [29]:
user_ratings_pivot = user_ratings_pivot.fillna(0)
user_ratings_pivot.loc[43463].sort_values(ascending=False).head(20)

title
Cosmopolis                                                 1.681818
It Follows                                                 1.181818
Maleficent                                                 1.181818
The Strange Color of Your Body's Tears                     1.181818
The Turin Horse                                            1.181818
The Tale of the Princess Kaguya                            1.181818
Milk                                                       1.181818
Blancanieves                                               1.181818
A Girl Walks Home Alone at Night                           1.181818
A Prophet                                                  1.181818
Once Upon a Time in Anatolia                               1.181818
Room 237                                                   0.681818
Tangerine                                                  0.681818
Behind the Candelabra                                      0.681818
Henri Georges Clouzot's Inferno           

In [30]:
user_ratings_pivot.loc[45811]

title
NaN                     -0.126689
!Women Art Revolution    0.000000
#1 Cheerleader Camp      0.000000
#Horror                  0.000000
#chicagoGirl             0.373311
                           ...   
Приключения Домовёнка    0.000000
Путь к себе              0.000000
Семь кабинок             0.000000
Юленька                  0.000000
هیچ کجا هیچ کس           0.000000
Name: 45811, Length: 14643, dtype: float64

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
cosine_similarity(user_ratings_pivot.loc[45811, :].values.reshape(1,-1), user_ratings_pivot.loc[43463, :].values.reshape(1,-1))

array([[0.01689601]])

In [33]:
user_similarities = cosine_similarity(user_ratings_pivot)
cosine_similarity_df = pd.DataFrame(user_similarities,index=user_ratings_pivot.index,columns=user_ratings_pivot.index)

In [34]:
cosine_similarity_df.head()

userId,20,79,111,166,231,242,273,332,340,363,...,270592,270628,270631,270634,270654,270720,270737,270757,270807,270887
userId,,,,,,,,,,,,,,,,,,,,,
20,1.000000,0.110973,0.016953,0.086861,0.012876,0.019616,0.011725,0.136135,0.000885,-0.079191,...,0.054698,0.033060,0.086589,-0.022291,0.091006,-0.010594,0.027381,-0.013433,0.086374,-0.009861
79,0.110973,1.000000,-0.000713,0.060761,-0.001415,-0.033535,-0.017502,-0.005323,-0.018612,-0.063375,...,0.036160,0.084464,-0.038710,0.001377,-0.040062,0.011631,0.006062,-0.004657,0.041905,-0.004695
111,0.016953,-0.000713,1.000000,0.041837,0.032916,0.072695,-0.055755,0.104979,-0.041718,0.003984,...,0.070865,-0.089603,0.042770,0.056968,-0.055323,-0.002321,0.081014,-0.098928,0.066176,0.007792
166,0.086861,0.060761,0.041837,1.000000,0.045658,0.019022,0.045120,0.076006,-0.078292,-0.058056,...,-0.021119,-0.075676,-0.034825,-0.038380,0.020789,0.100960,-0.066892,0.076809,0.140047,-0.024791
231,0.012876,-0.001415,0.032916,0.045658,1.000000,0.029650,-0.016223,-0.012930,-0.009620,0.033545,...,0.011586,-0.037406,0.020367,0.008709,-0.057739,0.008899,0.020568,-0.041425,0.037395,-0.019797


In [35]:
#tiding it up in a dataframe with the row and column names as the Ids of the users
cosine_similarity_df = pd.DataFrame(user_similarities, index=user_ratings_pivot.index,columns=user_ratings_pivot.index).fillna(0)

In [36]:
cosine_similarity_df.head()

userId,20,79,111,166,231,242,273,332,340,363,...,270592,270628,270631,270634,270654,270720,270737,270757,270807,270887
userId,,,,,,,,,,,,,,,,,,,,,
20,1.000000,0.110973,0.016953,0.086861,0.012876,0.019616,0.011725,0.136135,0.000885,-0.079191,...,0.054698,0.033060,0.086589,-0.022291,0.091006,-0.010594,0.027381,-0.013433,0.086374,-0.009861
79,0.110973,1.000000,-0.000713,0.060761,-0.001415,-0.033535,-0.017502,-0.005323,-0.018612,-0.063375,...,0.036160,0.084464,-0.038710,0.001377,-0.040062,0.011631,0.006062,-0.004657,0.041905,-0.004695
111,0.016953,-0.000713,1.000000,0.041837,0.032916,0.072695,-0.055755,0.104979,-0.041718,0.003984,...,0.070865,-0.089603,0.042770,0.056968,-0.055323,-0.002321,0.081014,-0.098928,0.066176,0.007792
166,0.086861,0.060761,0.041837,1.000000,0.045658,0.019022,0.045120,0.076006,-0.078292,-0.058056,...,-0.021119,-0.075676,-0.034825,-0.038380,0.020789,0.100960,-0.066892,0.076809,0.140047,-0.024791
231,0.012876,-0.001415,0.032916,0.045658,1.000000,0.029650,-0.016223,-0.012930,-0.009620,0.033545,...,0.011586,-0.037406,0.020367,0.008709,-0.057739,0.008899,0.020568,-0.041425,0.037395,-0.019797


In [37]:
cosine_similarity_series = cosine_similarity_df.loc[43463]

In [38]:
display(cosine_similarity_series)

userId
20        0.024941
79        0.019305
111      -0.009141
166      -0.053043
231      -0.034397
            ...   
270720    0.039728
270737   -0.021820
270757    0.038175
270807    0.013924
270887   -0.021963
Name: 43463, Length: 9725, dtype: float64

In [39]:
#we order it
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

In [40]:
display(ordered_similarities)

userId
43463     1.000000
15056     0.140513
96918     0.139813
89351     0.139040
244980    0.132495
            ...   
222926   -0.121224
239901   -0.123314
230332   -0.133554
185003   -0.136244
223755   -0.138054
Name: 43463, Length: 9725, dtype: float64

In [41]:
#pick the first 3
#KNN: find the most similar k neighbors to the usre in question, (here say k= 3) and then averages 
#the ratings those usres gave to the item we are trying to get a rating for..
#this gives us a predicted rating or how a user might feel about an item they haven't seen before
nearest_neighbors = ordered_similarities[1:4]

In [42]:
nearest_neighbors

userId
15056    0.140513
96918    0.139813
89351    0.139040
Name: 43463, dtype: float64

In [43]:
neighbor_ratings = user_ratings_table.reindex(nearest_neighbors.index)

In [44]:
neighbor_ratings

title,NaN,!Women Art Revolution,#1 Cheerleader Camp,#Horror,#chicagoGirl,$5 a Day,$50K and a Call Girl: A Love Story,$ellebrity,'71,'Til Madness Do Us Part,...,Іван Сила,Игра на выбывание,Каменная башка,Королёв,Крепость: щитом и мечом,Приключения Домовёнка,Путь к себе,Семь кабинок,Юленька,هیچ کجا هیچ کس
userId,,,,,,,,,,,,,,,,,,,,,
15056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
neighbor_ratings_non_empty = neighbor_ratings.fillna(0)
display(neighbor_ratings_non_empty)

title,NaN,!Women Art Revolution,#1 Cheerleader Camp,#Horror,#chicagoGirl,$5 a Day,$50K and a Call Girl: A Love Story,$ellebrity,'71,'Til Madness Do Us Part,...,Іван Сила,Игра на выбывание,Каменная башка,Королёв,Крепость: щитом и мечом,Приключения Домовёнка,Путь к себе,Семь кабинок,Юленька,هیچ کجا هیچ کس
userId,,,,,,,,,,,,,,,,,,,,,
15056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
def get_recommendations(user,title):
    cosine_similarity_series = cosine_similarity_df.loc[user]
    #we order it
    ordered_similarities = cosine_similarity_series.sort_values(ascending=False)
    #pick the first 3
    #KNN: find the most similar k neighbors to the usre in question, (here say k= 3) and then averages 
    #the ratings those usres gave to the item we are trying to get a rating for..
    #this gives us a predicted rating or how a user might feel about an item they haven't seen before
    nearest_neighbors = ordered_similarities[1:4]
    neighbor_ratings = user_ratings_table.reindex(nearest_neighbors.index)
    return neighbor_ratings[title].mean()

In [47]:
user_ratings_table.loc[43463,'Evil Dead']

nan

In [48]:
get_recommendations(43463,'Evil Dead')

4.5

In [49]:
target_user = 43463

In [50]:
target_user_x = user_ratings_pivot.loc[[target_user]]

In [51]:
print(target_user_x)

title   NaN  !Women Art Revolution  #1 Cheerleader Camp  #Horror  \
userId                                                             
43463   0.0                    0.0                  0.0      0.0   

title   #chicagoGirl  $5 a Day  $50K and a Call Girl: A Love Story  \
userId                                                               
43463            0.0       0.0                                 0.0   

title   $ellebrity  '71  'Til Madness Do Us Part  ...  Іван Сила  \
userId                                            ...              
43463          0.0  0.0                      0.0  ...        0.0   

title   Игра на выбывание  Каменная башка  Королёв  Крепость: щитом и мечом  \
userId                                                                        
43463                 0.0             0.0      0.0                      0.0   

title   Приключения Домовёнка  Путь к себе  Семь кабинок  Юленька  \
userId                                                              
434

In [52]:
#origina raw ratings for the item we are predicting
other_users_y = user_ratings_table["Up"]

In [53]:
print(other_users_y)

userId
20        4.5
79        3.5
111       4.0
166       3.0
231       3.0
         ... 
270720    2.5
270737    3.5
270757    NaN
270807    NaN
270887    5.0
Name: Up, Length: 9725, dtype: float64


In [54]:
other_users_x = user_ratings_pivot[other_users_y.notnull()]
print(other_users_x)

title   NaN  !Women Art Revolution  #1 Cheerleader Camp  #Horror  \
userId                                                             
20      0.0                    0.0                  0.0      0.0   
79      0.0                    0.0                  0.0      0.0   
111     0.0                    0.0                  0.0      0.0   
166     0.0                    0.0                  0.0      0.0   
231     0.0                    0.0                  0.0      0.0   
...     ...                    ...                  ...      ...   
270634  0.0                    0.0                  0.0      0.0   
270654  0.0                    0.0                  0.0      0.0   
270720  0.0                    0.0                  0.0      0.0   
270737  0.0                    0.0                  0.0      0.0   
270887  0.0                    0.0                  0.0      0.0   

title   #chicagoGirl  $5 a Day  $50K and a Call Girl: A Love Story  \
userId                                       

In [55]:
#similary drop the rows in the ratings that are empty
other_users_y.dropna(inplace=True)
print(other_users_y)

userId
20        4.5
79        3.5
111       4.0
166       3.0
231       3.0
         ... 
270634    1.0
270654    3.5
270720    2.5
270737    3.5
270887    5.0
Name: Up, Length: 6731, dtype: float64


In [56]:
from sklearn.neighbors import KNeighborsRegressor

In [57]:
user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=3)

In [58]:
#we fit it the same way we fit any model running it on our users and the raitngs for the specific book 
user_knn.fit(other_users_x, other_users_y)

KNeighborsRegressor(metric='cosine', n_neighbors=3)

In [59]:
user_user_pred = user_knn.predict(target_user_x)

In [60]:
print(user_user_pred)

[3.83333333]


In [61]:
get_recommendations(43463,'Up')

4.5